<a href="https://colab.research.google.com/github/NakaylaMinott/Intelligent-Music-Recommendation-and-Song-Similarity-System/blob/main/songSimilarityModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Direct FMA_small dataset download**

In [ ]:
# paste the official URL for fma_small.zip
FMA_SMALL_ZIP_URL = "https://<official-source>/fma_small.zip"

!wget -q --show-progress "https://os.unil.cloud.switch.ch/fma/fma_small.zip" -O /content/fma_small.zip
!unzip -q /content/fma_small.zip -d /content/fma_small


/content/fma_small. 100%[===================>]   7.15G  12.9MB/s    in 9m 39s  


In [ ]:
!mv /content/fma_small/fma_small/* /content/fma_small/
!ls /content/fma_small | head

mv: cannot move '/content/fma_small/fma_small/000' to '/content/fma_small/000': Directory not empty
mv: cannot move '/content/fma_small/fma_small/001' to '/content/fma_small/001': Directory not empty
mv: cannot move '/content/fma_small/fma_small/002' to '/content/fma_small/002': Directory not empty
mv: cannot move '/content/fma_small/fma_small/003' to '/content/fma_small/003': Directory not empty
mv: cannot move '/content/fma_small/fma_small/004' to '/content/fma_small/004': Directory not empty
mv: cannot move '/content/fma_small/fma_small/005' to '/content/fma_small/005': Directory not empty
mv: cannot move '/content/fma_small/fma_small/006' to '/content/fma_small/006': Directory not empty
mv: cannot move '/content/fma_small/fma_small/007' to '/content/fma_small/007': Directory not empty
mv: cannot move '/content/fma_small/fma_small/008' to '/content/fma_small/008': Directory not empty
mv: cannot move '/content/fma_small/fma_small/009' to '/content/fma_small/009': Directory not empty


# **Mount Google drive and load datasets**

In [ ]:
from pathlib import Path
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#install readers
!pip -q install pyarrow fastparquet

#point to dataset folder
base = Path("/content/drive/My Drive/senior project/datasets")
Parquet = base / "ab_fma_filtered.parquet"
FMA_meta_zip =  base / "fma_metadata.zip"

print(Parquet.exists(), Parquet)
print(FMA_meta_zip.exists(), FMA_meta_zip)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
True /content/drive/My Drive/senior project/datasets/ab_fma_filtered.parquet
True /content/drive/My Drive/senior project/datasets/fma_metadata.zip


In [ ]:
!pip -q install numpy==2.0.2 librosa==0.10.1 soundfile faiss-cpu tqdm transformers==4.44.2 accelerate==0.34.2
!sudo apt-get -y install libsndfile1 > /dev/null


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed-ucxx-cu12 0.44.0 requires numba<0.62.0a0,>=0.59.1, but you have numba 0.62.1 which is incompatible.
dask-cuda 25.6.0 requires numba<0.62.0a0,>=0.59.1, but you have numba 0.62.1 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
cuml-cu12 25.6.0 requires numba<0.62.0a0,>=0.59.1, but you have numba 0.62.1 which is incompatible.
cudf-cu12 25.6.0 requires numba<0.62.0a0,>=0.59.1, but you have numba 0.62.1 which is incompatible.


In [ ]:
import sys, numpy as np, librosa, torch
print("python:", sys.version.split()[0])
print("numpy:", np.__version__)
print("librosa:", librosa.__version__)
print("torch:", torch.__version__)


python: 3.12.12
numpy: 1.26.4
librosa: 0.10.1
torch: 2.8.0+cu126


In [ ]:
ab_features = pd.read_parquet(Parquet)
ab_features.head(), ab_features.shape

(   track_id       ab_artist                                  ab_title  \
 0       534  Charles Manson                             I Can See You   
 1       535  Charles Manson                      Angels Fear to Tread   
 2       540  Charles Manson                          The Black Pirate   
 3       541  Charles Manson                            So We Go Again   
 4       542  Charles Manson  Self Is Eternal (What Our World Will Be)   
 
                                 ab_mbid         bpm bpm_confidence key_key  \
 0  44d98503-e039-4472-9f81-5c81e9262314  144.266220           None      G#   
 1  16ff3444-d4d5-43dc-8123-15a800862bc5   97.698425           None       B   
 2  1641544e-64e2-4a86-baf1-de02c342277f  135.865082           None      G#   
 3  56b4d29d-a35d-46c2-a07a-3f068fdb1a99  105.680603           None      G#   
 4  4f4eaffa-99f4-4f72-92ac-e1cec2810565  139.959534           None       E   
 
   key_scale  key_strength mood_acoustic mood_aggressive mood_happy mood_sad  

In [ ]:
#list file names inside fma zip
import zipfile

with zipfile.ZipFile(FMA_meta_zip, 'r') as z:
    print(z.namelist()[:20])  # show first 20 files inside



['fma_metadata/README.txt', 'fma_metadata/checksums', 'fma_metadata/not_found.pickle', 'fma_metadata/raw_genres.csv', 'fma_metadata/raw_albums.csv', 'fma_metadata/raw_artists.csv', 'fma_metadata/raw_tracks.csv', 'fma_metadata/tracks.csv', 'fma_metadata/genres.csv', 'fma_metadata/raw_echonest.csv', 'fma_metadata/echonest.csv', 'fma_metadata/features.csv']


In [ ]:
#open tracks.csv
with zipfile.ZipFile(FMA_meta_zip) as z:
    with z.open("fma_metadata/tracks.csv") as f:
        tracks = pd.read_csv(f, header=[0,1,2], index_col=0)
#check structure of tracks
tracks.columns.levels[0]
tracks.head(3)


album                                            \
                   comments         date_created        date_released   
track_id Unnamed: 1_level_2   Unnamed: 2_level_2   Unnamed: 3_level_2   
2                         0  2008-11-26 01:44:45  2009-01-05 00:00:00   
3                         0  2008-11-26 01:44:45  2009-01-05 00:00:00   
5                         0  2008-11-26 01:44:45  2009-01-05 00:00:00   

                                                                   \
                   engineer          favorites                 id   
track_id Unnamed: 4_level_2 Unnamed: 5_level_2 Unnamed: 6_level_2   
2                       NaN                  4                  1   
3                       NaN                  4                  1   
5                       NaN                  4                  1   

                                                                   \
                information            listens           producer   
track_id Unnamed: 7_level_2 Unnamed: 8_level_2 Unnamed: 9_level_2   
2                   <p></p>               6073                NaN   
3                   <p></p>               6073                NaN   
5                   <p></p>               6073                NaN   

                              ...               track                      \
                        tags  ...         information            interest   
track_id Unnamed: 10_level_2  ... Unnamed: 43_level_2 Unnamed: 44_level_2   
2                         []  ...                 NaN                4656   
3                         []  ...                 NaN                1470   
5                         []  ...                 NaN                1933   

                              \
               language_code   
track_id Unnamed: 45_level_2   
2                         en   
3                         en   
5                         en   

                                                             \
                                                    license   
track_id                                Unnamed: 46_level_2   
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...   
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...   

                                                                      \
                     listens            lyricist              number   
track_id Unnamed: 47_level_2 Unnamed: 48_level_2 Unnamed: 49_level_2   
2                       1293                 NaN                   3   
3                        514                 NaN                   4   
5                       1151                 NaN                   6   

                                                                      
                   publisher                tags               title  
track_id Unnamed: 50_level_2 Unnamed: 51_level_2 Unnamed: 52_level_2  
2                        NaN                  []                Food  
3                        NaN                  []        Electric Ave  
5                        NaN                  []          This World  

[3 rows x 52 columns]

In [ ]:
import re
# Flatten (if MultiIndex) and strip the bogus 3rd-header suffix like "__Unnamed: 26_level_2"
flat_cols = ['__'.join([str(x) for x in tup if pd.notna(x) and str(x)!=''])
             for tup in tracks.columns.to_flat_index()]
clean_cols = [re.sub(r'__Unnamed:.*$', '', c) for c in flat_cols]
tracks.columns = clean_cols

# sanity peek
tracks.columns[:20]


Index(['album__comments', 'album__date_created', 'album__date_released',
       'album__engineer', 'album__favorites', 'album__id',
       'album__information', 'album__listens', 'album__producer',
       'album__tags', 'album__title', 'album__tracks', 'album__type',
       'artist__active_year_begin', 'artist__active_year_end',
       'artist__associated_labels', 'artist__bio', 'artist__comments',
       'artist__date_created', 'artist__favorites'],
      dtype='object')

In [ ]:
#clean flattened headers

def clean_cols(cols):
  cleaned = []
  for c in cols:
    c = c.split("__Unnamed")[0]
    cleaned.append(c)
  return cleaned

tracks.columns = clean_cols(tracks.columns)
tracks.columns[:52]

Index(['album__comments', 'album__date_created', 'album__date_released',
       'album__engineer', 'album__favorites', 'album__id',
       'album__information', 'album__listens', 'album__producer',
       'album__tags', 'album__title', 'album__tracks', 'album__type',
       'artist__active_year_begin', 'artist__active_year_end',
       'artist__associated_labels', 'artist__bio', 'artist__comments',
       'artist__date_created', 'artist__favorites', 'artist__id',
       'artist__latitude', 'artist__location', 'artist__longitude',
       'artist__members', 'artist__name', 'artist__related_projects',
       'artist__tags', 'artist__website', 'artist__wikipedia_page',
       'set__split', 'set__subset', 'track__bit_rate', 'track__comments',
       'track__composer', 'track__date_created', 'track__date_recorded',
       'track__duration', 'track__favorites', 'track__genre_top',
       'track__genres', 'track__genres_all', 'track__information',
       'track__interest', 'track__language_cod

In [ ]:
# slim FMA view(what will be merged with AB and used for artist disjoint)
fma_slim = pd.DataFrame({
    "track_id": tracks.index.astype(int),
    "artist": tracks["artist__name"],
    "genre_top": tracks["track__genre_top"]
})
fma_slim.head(), fma_slim.shape


(    track_id      artist genre_top
 2          2        AWOL   Hip-Hop
 3          3        AWOL   Hip-Hop
 5          5        AWOL   Hip-Hop
 10        10   Kurt Vile       Pop
 20        20  Nicky Cook       NaN,
 (106574, 3))

In [ ]:
# load AB parquet and keep needed labels

ab = pd.read_parquet(Parquet)
ab_keep = ab.rename(columns= {"key_key": "key",
                              "key_scale": "mode"})[["track_id","bpm","key","mode","mood_acoustic","mood_aggressive",
    "mood_happy","mood_sad","danceability","voice_instrumental"]]


In [ ]:
#mini merge of FMA and AB
merged = ab_keep.merge(fma_slim, on="track_id", how = "left")
merged.head(), merged.shape

(   track_id         bpm key   mode mood_acoustic mood_aggressive mood_happy  \
 0       534  144.266220  G#  minor          None            None       None   
 1       535   97.698425   B  major          None            None       None   
 2       540  135.865082  G#  major          None            None       None   
 3       541  105.680603  G#  minor          None            None       None   
 4       542  139.959534   E  minor          None            None       None   
 
   mood_sad danceability voice_instrumental          artist genre_top  
 0     None         None               None  Charles Manson      Folk  
 1     None         None               None  Charles Manson      Folk  
 2     None         None               None  Charles Manson      Folk  
 3     None         None               None  Charles Manson      Folk  
 4     None         None               None  Charles Manson      Folk  ,
 (709, 12))

In [ ]:
#add audio_path from track_id (ned a resolvable ppath for feather extraction or audio snaity checks)

from pathlib import Path
audio_root = Path("/content/fma_small")

def id_to_Path(tid:int) -> str:
  return str(audio_root / f"{tid // 1000:03d}"/f"{tid:06d}.mp3")

merged["audio_path"] = merged["track_id"].astype(int).map(id_to_Path)



In [ ]:
#create a cononical_artist for slitting( prevents the same artist being split by spelling)
import unicodedata

def canoicalize(name:str) -> str:
  if pd.isna(name): return None
  s = name.lower().strip()
  s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")
  s = re.sub(r"\(.*?\)|\[.*?\]", "", s)              # remove (...) and [...]
  s = re.sub(r"\bfeat\.?|featuring\b.*", "", s)     # remove feat clauses
  s = s.replace("&", " and ")
  s = re.sub(r"\s+", " ", s).strip()
  return s

merged["canonical_artist"] = merged["artist"].map(canoicalize)

In [ ]:
#make artist-disjoint splits (80/10/10, fixed seed) prevents leakage
import numpy as np

rng = np.random.default_rng(1337)
artists = merged["canonical_artist"].dropna().unique()
rng.shuffle(artists)
n = len(artists)

train_as = set(artists[:int(0.8*n)])
test_as = set(artists[int(0.8*n):int(0.9*n)])
val_as = set(artists[int(0.9*n):])

def assign(a):
  if a in train_as: return "train"
  if a in val_as: return "val"
  return "test"

merged["split"] = merged["canonical_artist"].map(assign)



In [ ]:
import os
# 1) make the boolean flag
merged["exists"] = merged["audio_path"].apply(lambda p: isinstance(p, str) and os.path.exists(p))

# 2) quick sanity: how many files are actually present?
print(merged["exists"].value_counts(dropna=False))
print("Missing examples:\n", merged.loc[~merged["exists"], "audio_path"].head(5))

# 3) filter to only rows that have an on-disk MP3 (your FMA-small subset)
merged_small = merged.loc[merged["exists"]].copy()
merged_small.shape

exists
False    619
True      90
Name: count, dtype: int64
Missing examples:
 1    /content/fma_small/000/000535.mp3
3    /content/fma_small/000/000541.mp3
4    /content/fma_small/000/000542.mp3
5    /content/fma_small/000/000543.mp3
6    /content/fma_small/000/000545.mp3
Name: audio_path, dtype: object


(90, 16)

In [ ]:

# Recheck file existence for the filtered dataset
merged_small["exists"] = merged_small["audio_path"].apply(os.path.exists)

# Show summary
merged_small["exists"].value_counts(normalize=True)


,proportion
exists,
True,1.0


In [ ]:
merged_small["split"].value_counts()
merged_small.groupby("split")["canonical_artist"].nunique()


,canonical_artist
split,
test,5
train,49
val,6


In [ ]:
#sanity check (check silent failures)

# Disjointness
A_tr, A_v, A_te = [set(merged.loc[merged.split==s,"canonical_artist"]) for s in ["train","val","test"]]
assert A_tr.isdisjoint(A_v) and A_tr.isdisjoint(A_te) and A_v.isdisjoint(A_te)

# Coverage
merged.groupby("split")["track_id"].nunique()

# Quick genre coverage view
merged.pivot_table(index="genre_top", columns="split", values="track_id", aggfunc="nunique", fill_value=0).head()

# File existence spot-check (sample 10)
import os
merged.sample(10, random_state=0)["audio_path"].apply(lambda p: os.path.exists(p)).value_counts()


,count
audio_path,
False,8
True,2


# **V1 Manifest**

In [ ]:

df = merged_small.copy()  # (your earlier line)

def pick_col(frame, candidates, default=np.nan):
    for c in candidates:
        if c in frame.columns:
            return frame[c]
    return default

manifest = pd.DataFrame({
    "track_id": df["track_id"].astype(int),
    "artist":   pick_col(df, ["artist","track_artist","artist_name"]),
    "title":    pick_col(df, ["title","track_title","ab_title","fma_title"]),
    "genre_top":pick_col(df, ["genre_top","track_genre_top","genre"]),
    "path":     pick_col(df, ["audio_path","path"]),
    "split":    df["split"],
    "bpm":      pick_col(df, ["bpm","ab_bpm","bpm_bpm"]),
    "key":      pick_col(df, ["key","key_key","ab_key"]),
    "mode":     pick_col(df, ["mode","ab_mode"]),
})

# minimal cleaning
for c in ["artist","title","genre_top"]:
    manifest[c] = manifest[c].astype(str).str.strip()

# Verify a few paths exist (first 20)
exists_20 = sum(isinstance(p, str) and os.path.exists(p) for p in manifest["path"].head(20))
print(f"Existence check (first 20): {exists_20} of 20")

# Save to your datasets folder on Drive
manifest_path = "/content/drive/My Drive/senior project/datasets/mini_manifest.parquet"
manifest.to_parquet(manifest_path, index=False)
print("Wrote:", manifest_path, "rows:", len(manifest))

manifest.head(3)


Existence check (first 20): 20 of 20
Wrote: /content/drive/My Drive/senior project/datasets/mini_manifest.parquet rows: 90


,track_id,artist,title,genre_top,path,split,bpm,key,mode
0,534,Charles Manson,nan,Folk,/content/fma_small/000/000534.mp3,train,144.266220,G#,minor
2,540,Charles Manson,nan,Folk,/content/fma_small/000/000540.mp3,train,135.865082,G#,major
13,1082,M. Nguyen Van Minh-Con,nan,International,/content/fma_small/001/001082.mp3,train,142.756546,F,minor


In [ ]:

import torch, librosa, numpy as np
print("torch:", torch.__version__)
print("librosa:", librosa.__version__)
print("numpy:", np.__version__)


torch: 2.8.0+cu126
librosa: 0.10.1
numpy: 1.26.4


In [ ]:
import os, numpy as np, pandas as pd, torch, librosa
from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2Model

#load mainfest
manifest_path = "/content/drive/My Drive/senior project/datasets/mini_manifest.parquet"
df = pd.read_parquet(manifest_path)

#where embedding will be saved
emb_root = "/content/drive/My Drive/senior project/song-similarity-v1/data/embeddings"
os.makedirs(f"{emb_root}/train", exist_ok=True)
os.makedirs(f"{emb_root}/val",   exist_ok=True)
os.makedirs(f"{emb_root}/test",  exist_ok=True)

#load pretrained wav2vec2 model
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device).eval()

#embedding a single track (embed_w2v)
SR = 16000       # Wav2Vec2 expects 16 kHz
MAX_SEC = 30     # clip length for speed

@torch.inference_mode()
def embed_w2v(audio_path: str) -> np.ndarray:
    y, _ = librosa.load(audio_path, sr=SR, mono=True, duration=MAX_SEC)
    if y.size == 0:  # safety
        y = np.zeros(int(SR), dtype=np.float32)
    inputs = processor(y, sampling_rate=SR, return_tensors="pt", padding=False)
    out = model(inputs.input_values.to(device))
    # last_hidden_state: [1, T, 768] → mean over time → [768]
    vec = out.last_hidden_state.mean(dim=1).squeeze(0).detach().cpu().numpy().astype("float32")
    return vec

#looping over dataset and saving (1 .npy per track)
emb_paths = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    audio_path = row["path"]
    out_path = f"{emb_root}/{row['split']}/{int(row['track_id'])}.npy"
    try:
        vec = embed_w2v(audio_path)
        np.save(out_path, vec)
        emb_paths.append(out_path)
    except Exception as e:
        print("Failed:", row["track_id"], e)
        emb_paths.append("")

#new manifest writen with emb_path column
df["emb_path"] = emb_paths
man_out = "/content/drive/My Drive/senior project/song-similarity-v1/manifests/mini_manifest_with_emb.parquet"
os.makedirs(os.path.dirname(man_out), exist_ok=True)
df.to_parquet(man_out, index=False)
print("Saved:", man_out, "rows:", len(df))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning

Saved: /content/drive/My Drive/senior project/song-similarity-v1/manifests/mini_manifest_with_emb.parquet rows: 90


In [ ]:
import numpy as np, pandas as pd, os

man_out = "/content/drive/My Drive/senior project/song-similarity-v1/manifests/mini_manifest_with_emb.parquet"
df = pd.read_parquet(man_out)

print("rows:", len(df))
print("have vectors:", df["emb_path"].str.endswith(".npy").value_counts())

p = df.loc[df["emb_path"].str.endswith(".npy")].iloc[0]["emb_path"]
v = np.load(p)
print("vector shape:", v.shape, "dtype:", v.dtype)   # expect (768,), float32


rows: 90
have vectors: emb_path
True    90
Name: count, dtype: int64
vector shape: (768,) dtype: float32


# **Building FAISS**





In [ ]:
import faiss
from tqdm import tqdm

man_out = "/content/drive/My Drive/senior project/song-similarity-v1/manifests/mini_manifest_with_emb.parquet"
df = pd.read_parquet(man_out)
train = df[df["split"]=="train"].copy()

#load vectors & ids
X, ids = [], []
for j, r in train.iterrows():
  p = r